In [ ]:
from hashlib import sha1
import hmac
import binascii
def getUrl(request):
    devId = 3002228
    key = '5bef2099-6493-48e4-b344-e37e99530ce6'
    request = request + ('&' if ('?' in request) else '?')
    raw = request+'devid={0}'.format(devId)
    #print(raw.encode())
    key_bin = key.encode()
    raw_bin = raw.encode() 
    hashed = hmac.new(key_bin, raw_bin, sha1)
    signature = hashed.hexdigest()
    return 'http://timetableapi.ptv.vic.gov.au'+raw+'&signature={1}'.format(devId, signature)
#print(getUrl('/v3/route_types'))


import requests
import json
def download_get(url_name,file_path):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    with open(file_path,'w') as f:
        f.write(response.text)
def download_get_to_dict(url_name):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    return json.loads(response.text)


routes_json_file = '../data/routes.json'
with open(routes_json_file) as f:
    route_dict = json.load(f)
    stops_files = [open(f'../data/stops_file_{i}.json','a')  for i in range(5)]
    for stop_file in stops_files:
        stop_file.write('[\n')
    is_first_line = [True,True,True,True,True]
    for route_item in route_dict['routes']:
        route_id = route_item['route_id']
        route_type = route_item['route_type']
        url_name = f'/v3/stops/route/{route_id}/route_type/{route_type}'
        res = download_get_to_dict(url_name)
        stops = res['stops']
        for stop in stops:
            if not is_first_line[route_type]:
                stops_files[route_type].write(',\n')
            json.dump(stop,stops_files[route_type])
            is_first_line[route_type] = False
    
    for stop_file in stops_files:
        stop_file.write('\n]')
        stop_file.close()  

In [188]:
def to_lower(text):
    text = text.lower()
    return text

In [529]:
import pandas as pd
train_stops_file = pd.read_json('../data/stops_file_0.json')
train_stops_file = train_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
train_stops_file['stop_suburb'] = train_stops_file['stop_suburb'].apply(to_lower)
train_stops_file['stop_name'] = train_stops_file['stop_name'].drop_duplicates()

In [530]:
train_stops_file = train_stops_file.dropna()
train_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ashburton,-37.868317,145.079666,1002,Alamein Station
1,ashburton,-37.861970,145.081345,1010,Ashburton Station
2,hawthorn east,-37.822400,145.045837,1012,Auburn Station
3,burnley,-37.827557,145.007553,1030,Burnley Station
4,glen iris,-37.851562,145.080500,1031,Burwood Station
...,...,...,...,...,...
347,williams landing,-37.869870,144.747452,1225,Williams Landing Station
348,yarraville,-37.815850,144.889938,1216,Yarraville Station
355,williamstown,-37.857330,144.889069,1146,North Williamstown Station
361,williamstown,-37.867756,144.905319,1211,Williamstown Station


In [197]:
def change_name(text):
    if(text == 'melbourne city'):
        return 'melbourne'
    else:
        return text
train_stops_file['stop_suburb'] = train_stops_file['stop_suburb'].apply(change_name)

In [364]:
tram_stops_file = pd.read_json('../data/stops_file_1.json')
tram_stops_file = tram_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
tram_stops_file['stop_suburb'] = tram_stops_file['stop_suburb'].apply(to_lower)
tram_stops_file['stop_name'] = tram_stops_file['stop_name'].drop_duplicates()
tram_stops_file['stop_suburb'] = tram_stops_file['stop_suburb'].apply(change_name)
tram_stops_file = tram_stops_file.dropna()
tram_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,brunswick east,-37.769787,144.971954,2227,Albert St/Lygon St #123
1,brunswick east,-37.761890,144.974060,2240,Albion St/Holmes St #127
2,brunswick east,-37.762370,144.973267,2231,Albion St/Lygon St #127
3,southbank,-37.821842,144.969513,2203,Arts Precinct/St Kilda Rd #14
4,southbank,-37.823814,144.967255,2266,Arts Precinct/Sturt St #17
...,...,...,...,...,...
1509,prahran,-37.850563,144.984039,2978,Punt Rd/High St #28
1516,coburg,-37.755560,144.964279,2236,Sydney Rd/Moreland Rd #132
1517,prahran,-37.852440,144.998100,2640,The Avenue/High St #33
1519,glen iris,-37.858078,145.041046,2622,Tooronga Rd/High St #48


In [365]:
bus_stops_file = pd.read_json('../data/stops_file_2.json')
bus_stops_file = bus_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
bus_stops_file['stop_suburb'] = bus_stops_file['stop_suburb'].apply(to_lower)
bus_stops_file['stop_name'] = bus_stops_file['stop_name'].drop_duplicates()
bus_stops_file['stop_suburb'] = bus_stops_file['stop_suburb'].apply(change_name)
bus_stops_file = bus_stops_file.dropna()
bus_stops_file 

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,avondale heights,-37.762234,144.859680,23570,Avondale Heights Reserve/Doyle St
1,maribyrnong,-37.769444,144.872772,31671,Canning Reserve/Canning St
3,avondale heights,-37.769413,144.857254,25271,Canning St/Herbert St
4,maribyrnong,-37.769013,144.877823,11239,Central Park Ave/Cordite Ave
5,avondale heights,-37.759260,144.857391,25277,Davis Ave/Skewes St
...,...,...,...,...,...
29797,hampton park,-38.030907,145.258270,19099,Robert Booth Reserve/Somerville Rd
29798,hampton park,-38.042637,145.270248,17513,Robert Ct/Huntington Dr
29799,narre warren south,-38.050587,145.296158,17528,Seebeck Dr/Sneddon Dr
29802,hampton park,-38.039837,145.280609,17583,The Ridge/Ralph Cres


In [366]:
vline_stops_file = pd.read_json('../data/stops_file_3.json')
vline_stops_file = vline_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
vline_stops_file['stop_suburb'] = vline_stops_file['stop_suburb'].apply(to_lower)
vline_stops_file['stop_name'] = vline_stops_file['stop_name'].drop_duplicates()
vline_stops_file['stop_suburb'] = vline_stops_file['stop_suburb'].apply(change_name)
vline_stops_file = vline_stops_file.dropna()
vline_stops_file 

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ararat,-37.282440,142.936900,1501,Ararat Railway Station
1,ararat,-37.283640,142.931992,26759,Ararat Town Hall/Barkly St
2,macarthur,-38.033012,141.999741,21122,Eversley St/Hamilton - Port Fairy Rd
3,hamilton,-37.746600,142.028275,4125,Hamilton Station/Station St
4,dunkeld,-37.649464,142.342560,4130,Information Centre/Glenelg Hwy
...,...,...,...,...,...
1381,alexandra,-37.190964,145.709076,26124,Perkiins St/Grant St
1383,trawool,-37.093380,145.204620,28302,Upper Goulburn Rd/Goulburn Valley Hwy
1391,clunes,-37.303142,143.781570,1591,Clunes Railway Station
1405,talbot,-37.172900,143.705627,1594,Talbot Railway Station


In [ ]:
from urllib import parse
import html2text as html2text
import requests
import json
url = "https://vahi.vic.gov.au/views/ajax?_wrapper_format=drupal_ajax"
for i in range(1):
    post_body = parse.urlencode({"view_name":"hospital_and_health_services","view_display_id":"page_1","page":i})
    response = requests.post(url= url,headers={
        "accept": "application/json, text/javascript, */*; q=0.01",
        "accept-language": "zh-CN,zh;q=0.9",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "sec-ch-ua": "\"Chromium\";v=\"104\", \" Not A;Brand\";v=\"99\", \"Google Chrome\";v=\"104\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "x-requested-with": "XMLHttpRequest"
      },data = post_body)
    print(response.status_code)
    print(response.text)
    with open(f'../data/v2_ajax_download_{i:02d}','w') as f:
        f.write(response.text)



with open('../data/v2_ajax_download_00') as f:
    d = json.load(f)
    print(len(d))
    geofield_google_map = d[0]["settings"]["geofield_google_map"]
    with open('../data/v2_ajax_download.csv','w') as csv_file:
        csv_file.write('"name","address"\n')
        for geofield in geofield_google_map:
            for feature in geofield_google_map[geofield]["data"]["features"]:
                #print(feature)
                name_with_tag = feature["properties"]["data"]["view_node"]
                name = name_with_tag[name_with_tag.index('>')+1 : name_with_tag.index('<',1)]
                address = feature["properties"]["data"]["field_address"]
                #print(html2text.html2text(),end=";")
                csv_file.write(f'"{name}","{address}"\n')

In [518]:
hospital = pd.read_csv("../data/v2_ajax_download.csv")


In [519]:
for i in range(len(hospital)):
    postcode_text = re.findall(r'VIC [0-9]+',hospital['address'][i])
    if len(postcode_text) == 1:
        hospital.loc[i,'postcode'] = postcode_text[0].split(' ')[1]
    elif len(postcode_text) == 0:
        hospital.loc[i,'postcode'] = None
    else:
        hospital.loc[i,'postcode'] = postcode_text[0].split(' ')[1]



In [521]:
hospital = hospital.dropna().reset_index()
hospital = hospital.drop('index',axis = 1)
hospital.to_csv('../data/hospital_csv')

In [355]:
geo_df = pd.read_excel("../data/postcode_in_VIC.csv")
geo_df = geo_df[geo_df['Postcode'] > 0 ]
geo_df = geo_df[geo_df['Postcode'] < 8000 ]
geo_df['Locality'] = geo_df['Locality'].apply(to_lower)


In [371]:
import math
EARTH_REDIUS = 6378.137


def rad(d):
    return d * math.pi / 180.0


def getDistance(lat1, lng1, lat2, lng2):
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * math.asin(math.sqrt(math.pow(math.sin(a/2), 2) + math.cos(radLat1) * math.cos(radLat2) * math.pow(math.sin(b/2), 2)))
    s = s * EARTH_REDIUS
    return s


def find_postcode(geo_df,lat, lng, suburb):
    if(suburb == 'melbourne'):
        distance = 10000
        result = None
        for index, item in geo_df.iterrows():
            postcode = item['Postcode']
            lat2, lng2 = item['Latitude'], item['Longitude']
            d = getDistance(lat, lng, lat2, lng2)
            if (d < distance):
                distance = d
                result = postcode
        return int(result)
    else:
        for index, item in geo_df.iterrows():
            locality = item['Locality']
            if locality == suburb:
                return int(item['Postcode'])


In [531]:
train_stops_file['postcode'] = None
for index, item in train_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    train_stops_file.loc[index, 'postcode'] = postcode


In [535]:
train_stops_file = train_stops_file.reset_index().drop('index', axis = 1)

In [372]:
tram_stops_file['postcode'] = None
for index, item in tram_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    tram_stops_file.loc[index, 'postcode'] = postcode
    

In [536]:
tram_stops_file = tram_stops_file.reset_index().drop('index', axis = 1)

In [373]:
bus_stops_file['postcode'] = None
for index, item in bus_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    bus_stops_file.loc[index, 'postcode'] = postcode

In [533]:
bus_stops_file = bus_stops_file.reset_index().drop('index', axis = 1)

In [374]:
vline_stops_file['postcode'] = None
for index, item in vline_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    vline_stops_file.loc[index, 'postcode'] = postcode

In [538]:
vline_stops_file = vline_stops_file.reset_index().drop('index', axis = 1)
vline_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name,postcode
0,ararat,-37.282440,142.936900,1501,Ararat Railway Station,3377
1,ararat,-37.283640,142.931992,26759,Ararat Town Hall/Barkly St,3377
2,macarthur,-38.033012,141.999741,21122,Eversley St/Hamilton - Port Fairy Rd,3286
3,hamilton,-37.746600,142.028275,4125,Hamilton Station/Station St,3300
4,dunkeld,-37.649464,142.342560,4130,Information Centre/Glenelg Hwy,3294
...,...,...,...,...,...,...
634,alexandra,-37.190964,145.709076,26124,Perkiins St/Grant St,3714
635,trawool,-37.093380,145.204620,28302,Upper Goulburn Rd/Goulburn Valley Hwy,3660
636,clunes,-37.303142,143.781570,1591,Clunes Railway Station,3370
637,talbot,-37.172900,143.705627,1594,Talbot Railway Station,3371


In [539]:
train_stops_file.to_csv('../data/train_stops_file.csv')
tram_stops_file.to_csv('../data/tram_stops_file.csv')
bus_stops_file.to_csv('../data/bus_stops_file.csv')
vline_stops_file.to_csv('../data/vline_stops_file.csv')

In [1]:
import pandas as pd
import openrouteservice
import math
house_df_combined = pd.read_csv('../data/house_combined.csv')
import ast
lat_list = []
long_list = []
for i in range(len(house_df_combined)):
    if pd.isnull(house_df_combined.geoLocation[i]):
        lat_list.append(None)
        long_list.append(None)
    else:
        latitude = ast.literal_eval(house_df_combined.geoLocation[i])['latitude']
        longitude = ast.literal_eval(house_df_combined.geoLocation[i])['longitude']
        lat_list.append(latitude)
        long_list.append(longitude)
house_df_combined['latitude'] = lat_list
house_df_combined['longitude'] = long_list

In [2]:
distance_list_6 = []
for i in range(74000,76000):
    print(i)
    if math.isnan(house_df_combined['longitude'][i]) is True:
        distance_list_6.append(None)
    else:
        coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
        client = openrouteservice.Client(key='5b3ce3597851110001cf6248f7b00ebe5e03493a9bee51dbc79e11b5') 
        routes = client.directions(coords, profile='driving-car', optimize_waypoints=True, radiuses = 1000)
        distance = routes['routes'][0]['summary']['distance']
        print(distance)
        distance_list_6.append(distance)

74000


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/directions.py:202: UserWarning: Less than 4 coordinates, nothing to optimize!
  warnings.warn("Less than 4 coordinates, nothing to optimize!", UserWarning)


73476.7
74001
85862.6
74002
65783.8
74003
74663.9
74004
78998.2
74005
70823.4
74006
67059.8
74007
86040.8
74008
82759.6
74009
84837.8
74010
79253.7
74011
86503.4
74012
83724.2
74013
67482.3
74014
79253.7
74015
66949.0
74016
82865.0
74017
83723.3
74018
72793.8
74019
82498.3
74020
80871.5
74021
74833.6
74022
88901.8
74023
83763.1
74024
75411.9
74025
78496.8
74026
78409.6
74027
82396.4
74028
80080.9
74029
75297.1
74030
79126.8
74031
92774.2
74032
72613.3
74033
78743.8
74034
64957.1
74035
76812.6
74036
91071.6
74037
66855.1
74038
81449.7
74039
103057.3
74040


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyon

106847.4
74041
80336.7
74042
114149.1
74043
81682.6
74044
87962.8
74045
81719.6
74046
71911.6
74047
82493.7
74048
75716.8
74049
76817.7
74050
100682.2
74051
87971.7
74052
68716.2
74053
88634.8
74054
90629.8
74055
76556.5
74056
102661.3
74057
78484.2
74058
91704.5
74059
74944.2
74060
74449.2
74061
85000.6
74062
80441.6
74063
89479.7
74064
90491.5
74065
92677.6
74066
90266.1
74067
92456.3
74068
91129.2
74069
89968.6
74070
90264.9
74071
92036.2
74072
89160.7
74073
89160.7
74074
91085.8
74075
78773.1
74076
85344.9
74077
90692.3
74078
93006.7
74079
102863.6
74080
88766.0
74081
72013.6
74082
73434.6
74083
89099.5
74084
103029.1
74085
90102.3
74086
136187.3
74087
86328.9
74088
90042.7
74089
72013.6
74090
89562.7
74091
88466.8
74092
92683.3
74093
90691.2
74094
91019.6
74095
90707.7
74096
71710.9
74097
80387.0
74098
90112.3
74099
88747.1
74100
90697.9
74101
90629.8
74102
87040.7
74103
94639.4
74104
102747.0
74105
101007.7
74106
98256.8
74107
76312.7
74108
89828.8
74109
94005.6
74110
90954.9
741

/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 8th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


73594.1
74251
80124.7
74252
77281.9
74253
12466.8
74254
7985.0
74255
36479.9
74256
77205.4
74257
78455.4
74258
55336.8
74259
72481.7
74260
78103.2
74261
80845.9
74262
77214.9
74263
85975.6
74264
82880.6
74265
75444.2
74266
83160.0
74267
102149.7
74268
75398.9
74269
69864.1
74270
66596.6
74271
106124.1
74272
87004.6
74273
77474.2
74274
83216.9
74275
75449.0
74276
79386.9
74277
101689.3
74278
108626.5
74279
86568.1
74280
75363.8
74281
1140.1
74282
108282.1
74283
80845.9
74284
101050.6
74285
108282.1
74286
102430.4
74287
85975.6
74288
101322.8
74289
75387.0
74290
78451.8
74291
76354.1
74292
102672.2
74293
77281.9
74294
72481.7
74295
102149.7
74296
75398.9
74297
79304.5
74298
78451.8
74299
81904.8
74300
101015.9
74301
80060.3
74302
81321.6
74303
111648.6
74304
107921.1
74305
101290.9
74306
112547.6
74307
86194.1
74308
87350.6
74309
75346.3
74310
75625.9
74311
102500.0
74312
80120.5
74313
87455.6
74314
78216.7
74315
105776.5
74316
75441.4
74317
81442.5
74318
80427.6
74319
101350.0
74320
813

ProxyError: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: /v2/directions/driving-car/json (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))

In [3]:
distance_list_7 = []
for i in range(75759,76000):
    print(i)
    if math.isnan(house_df_combined['longitude'][i]) is True:
        distance_list_7.append(None)
    else:
        coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
        client = openrouteservice.Client(key='5b3ce3597851110001cf6248f7b00ebe5e03493a9bee51dbc79e11b5') 
        routes = client.directions(coords, profile='driving-car', optimize_waypoints=True, radiuses = 1000)
        distance = routes['routes'][0]['summary']['distance']
        print(distance)
        distance_list_7.append(distance)

75759


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/directions.py:202: UserWarning: Less than 4 coordinates, nothing to optimize!
  warnings.warn("Less than 4 coordinates, nothing to optimize!", UserWarning)


63993.3
75760
60461.2
75761
61819.0
75762
63913.7
75763
62956.6
75764
66168.2
75765
58847.7
75766
59782.6
75767
63300.5
75768
62885.6
75769
63456.9
75770
64143.5
75771
57765.7
75772
63243.7
75773
65949.9
75774
64122.8
75775
59031.7
75776
63556.2
75777
63556.2
75778
63880.9
75779
82183.1
75780
60890.7
75781
65420.1
75782
62924.7
75783
59303.4
75784
48884.0
75785
62912.3
75786
51460.1
75787
62895.5
75788
62446.3
75789
77675.2
75790
75391.1
75791
78916.2
75792
75427.9
75793
84939.9
75794
76845.6
75795
81496.1
75796
79708.3
75797
75675.2
75798
91526.9
75799


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyon

100822.7
75800
75585.5
75801
82301.6
75802
83460.2
75803
90084.7
75804
75393.8
75805
76070.4
75806
74170.5
75807
79113.8
75808
79071.7
75809
81851.3
75810
102805.2
75811
66527.9
75812
86037.2
75813
82902.6
75814
77181.6
75815
75270.8
75816
82681.3
75817
75517.0
75818
73630.6
75819
64986.5
75820
74772.6
75821
87545.8
75822
80744.1
75823
69228.4
75824
63659.8
75825
70918.3
75826
73855.6
75827
66507.9
75828
74661.0
75829
29744.4
75830
70308.6
75831
71332.9
75832
86131.6
75833
82115.1
75834
62426.6
75835
73855.6
75836
79982.0
75837
75746.3
75838
75551.6
75839
74642.1
75840
79991.7
75841
78514.9
75842
102805.2
75843
74017.1
75844
62713.8
75845
78514.9
75846
79406.3
75847
79065.6
75848
71180.9
75849
78670.0
75850
78514.9
75851
65698.6
75852
70273.2
75853
69228.4
75854
74017.1
75855
73630.6
75856
90597.4
75857
78670.0
75858
79071.7
75859
75388.6
75860


ProxyError: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: /v2/directions/driving-car/json (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))

In [4]:
distance_list_8 = []
for i in range(75860,76000):
    print(i)
    if math.isnan(house_df_combined['longitude'][i]) is True:
        distance_list_8.append(None)
    else:
        coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
        client = openrouteservice.Client(key='5b3ce3597851110001cf6248f7b00ebe5e03493a9bee51dbc79e11b5') 
        routes = client.directions(coords, profile='driving-car', optimize_waypoints=True, radiuses = 1000)
        distance = routes['routes'][0]['summary']['distance']
        print(distance)
        distance_list_8.append(distance)

75860


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/directions.py:202: UserWarning: Less than 4 coordinates, nothing to optimize!
  warnings.warn("Less than 4 coordinates, nothing to optimize!", UserWarning)


70278.1
75861
81930.5
75862
70278.2
75863
75388.6
75864
73855.6
75865
73572.9
75866
78192.1
75867
64180.7
75868
68047.6
75869
76410.1
75870
81931.1
75871
81931.1
75872
81931.1
75873
82681.3
75874
80783.6
75875
64927.8
75876
68879.9
75877
64927.8
75878
74336.8
75879
67855.5
75880
66922.1
75881
76385.3
75882
82694.7
75883
74494.8
75884
84248.2
75885
70278.1
75886
77072.0
75887
75914.4
75888
75270.8
75889
114813.6
75890
115697.7
75891
116793.3
75892
118501.7
75893
116014.3
75894
155166.6
75895
149284.2
75896
155766.6
75897
151975.1
75898
159048.9
75899
151551.4
75900


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyon

153862.0
75901
156953.4
75902
153038.7
75903
159331.3
75904
154254.2
75905
153209.4
75906
178835.9
75907
155141.0
75908
155566.0
75909
152671.5
75910
152575.1
75911
159847.9
75912
151779.9
75913
149825.7
75914
159048.9
75915
157223.9
75916
152891.4
75917
154497.1
75918
153989.8
75919
146333.0
75920
153671.7
75921
152732.3
75922
154360.1
75923
158583.0
75924
170010.6
75925
151870.8
75926
146253.9
75927
146333.0
75928
153082.5
75929
153757.7
75930
153623.5
75931
149039.5
75932
152575.1
75933
150558.6
75934
151394.0
75935
150974.0
75936
156942.0
75937
153154.2
75938
152393.5
75939
152671.5
75940
153266.2
75941
153012.5
75942
153209.4
75943
154251.3
75944
152943.8
75945
155104.8
75946
152630.8
75947
154274.4
75948
155534.0
75949
160524.2
75950
154138.6
75951
158811.9
75952
153788.5
75953
165218.2
75954
152732.3
75955
152646.3
75956
154698.4
75957
155141.0
75958
155441.7
75959
150816.8
75960
152147.0
75961
162782.1
75962
154645.1
75963
150816.8
75964
153154.2
75965
145290.2
75966
151394.0
7

ApiError: 403 ({'error': 'Quota exceeded'})

In [6]:
distance_list_9 = []
for i in range(75999,76000):
    print(i)
    if math.isnan(house_df_combined['longitude'][i]) is True:
        distance_list_9.append(None)
    else:
        coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
        client = openrouteservice.Client(key='5b3ce3597851110001cf6248d883feabd2814b59bd5f07022d1a2f49') 
        routes = client.directions(coords, profile='driving-car', optimize_waypoints=True, radiuses = 1000)
        distance = routes['routes'][0]['summary']['distance']
        print(distance)
        distance_list_9.append(distance)

75999


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/directions.py:202: UserWarning: Less than 4 coordinates, nothing to optimize!
  warnings.warn("Less than 4 coordinates, nothing to optimize!", UserWarning)


153022.2


In [8]:
pd.DataFrame(distance_list_6+distance_list_7+distance_list_8+distance_list_9).to_csv('../data/raw/distance_list42')